In [103]:

import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler



In [104]:
dTrain = pd.read_csv('train.csv')
dTest = pd.read_csv('test.csv')
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [105]:
def extractHP(model):
	if 'HP' in model.upper():
		return model.split('HP')[0]
	return None

dTest['horse_power'] = dTest['engine'].apply(extractHP)

dTrain['horse_power'] = dTrain['engine'].apply(extractHP)
dTrain

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horse_power
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,172.0
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,320.0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,420.0
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,385.0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900,469.0
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900,None


In [106]:
import re

def extractEngineCapacity(model):
	ec = re.search("\d+\.?\d*L", model)
	return ec[0][:-1] if ec != None else None

dTest['engine_capacity'] = dTest['engine'].apply(extractEngineCapacity)

dTrain['engine_capacity'] = dTrain['engine'].apply(extractEngineCapacity)
dTrain

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_30913/2237485727.py:4: SyntaxWarning: invalid escape sequence '\d'
  ec = re.search("\d+\.?\d*L", model)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horse_power,engine_capacity
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,420.0,6.2
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,385.0,3.0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900,469.0,4.0
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900,None,3.0


In [107]:
from datetime import datetime

dTest['model_age'] = datetime.now().year - dTest['model_year']

dTrain['model_age'] = datetime.now().year - dTrain['model_year']

In [108]:
test_status = pd.get_dummies(dTest['accident'], prefix='status')
train_status = pd.get_dummies(dTrain['accident'], prefix='status')

dTest['accident']= test_status['status_At least 1 accident or damage reported'].apply(lambda x: True if x else False)
dTrain['accident']= train_status['status_At least 1 accident or damage reported'].apply(lambda x: True if x else False)

In [109]:
dTrain

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horse_power,engine_capacity,model_age
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,False,Yes,4200,172.0,1.6,17
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,True,Yes,4999,252.0,3.9,22
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,False,Yes,13900,320.0,5.3,22
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,False,Yes,45000,420.0,5.0,7
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,False,Yes,97500,208.0,2.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,False,Yes,27500,420.0,6.2,7
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,True,Yes,30000,385.0,3.0,6
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,False,Yes,86900,469.0,4.0,3
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,False,NaN,84900,None,3.0,2


In [110]:
from sklearn.preprocessing import LabelEncoder

toEncode = [
	"brand",
	"fuel_type",
	"ext_col",
	"int_col",
	"transmission",
]

encoder  = LabelEncoder()

for col in toEncode:
    dTest[f"{col}_label"] = encoder.fit_transform(dTest[col])
    dTrain[f"{col}_label"] = encoder.fit_transform(dTrain[col])
    


In [111]:
dTest

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,horse_power,engine_capacity,model_age,brand_label,fuel_type_label,ext_col_label,int_col_label,transmission_label
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,False,Yes,240.0,2.0,9,26,2,302,10,16
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,False,Yes,395.0,3.0,4,26,3,261,14,31
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,False,NaN,None,3.5,2,14,2,302,57,3
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,False,NaN,None,None,8,3,2,259,14,39
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,False,Yes,252.0,2.0,6,3,2,127,14,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125685,314218,Mercedes-Benz,GL-Class GL 450 4MATIC,2014,83315,Gasoline,362.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,False,Yes,362.0,3.0,10,36,2,29,14,23
125686,314219,Audi,Q7 55 Prestige,2019,29336,Gasoline,3.0 Liter Turbo,Automatic,White,Black,False,NaN,None,None,5,3,2,302,14,39
125687,314220,Audi,A6 3.0T Premium Plus,2012,77634,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,False,Yes,333.0,3.0,12,3,2,29,14,38
125688,314221,Audi,Q7 3.0T Premium,2012,112000,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,False,Yes,333.0,3.0,12,3,2,29,14,38


In [112]:
toImpute = [
	"horse_power",
	"engine_capacity"
]

for col in toImpute:
	imputer= SimpleImputer(strategy = 'mean')
	dTrain[col] = imputer.fit_transform(dTrain[[col]])
	dTest[col] = imputer.fit_transform(dTest[[col]])
	



In [113]:
toDrop = [
    "id",
	"clean_title",
	"model",
	"engine",
	"transmission",
	"model_year"
] + toEncode

# Removes duplicates
toDrop = list(dict.fromkeys(toDrop))
dte = dTest.drop(toDrop, axis = 1, inplace = True)
dtr = dTrain.drop(toDrop, axis = 1, inplace = True)


In [114]:
import warnings

# Ignore all FutureWarning messages
warnings.filterwarnings("ignore", category=FutureWarning)
import itertools
columns = dTrain.columns
result = []
for r in range(1,len(columns) + 1):
    for subset in itertools.combinations(columns, r):
        result.append(subset)
result = [list(comb) for comb in result]
i = 0
lowest_mse = 100000
lowest_result = 0
for rs in result:
    if i > 1000:
        break
    if rs and 'price' not in rs:
        rs.append('price')
        X = dTrain.drop(rs, axis=1)
        X = sm.add_constant(X)
        y = dTrain['price']
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=36)
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        model = LinearRegression()
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred, squared = False)
        if mse < lowest_mse:
            lowest_mse = mse
            lowest_result = rs
            print("lowest MSE: ", mse)
            print("rs : ", rs)

In [115]:
lowest = 100000
sf=pd.read_csv(sample_submission_url)
id = sf.pop('id')

for i in range(1, 100000):
    X = dTrain.drop(['transmission_label', 'price'], axis=1)
    X = sm.add_constant(X)
    y = dTrain['price']
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=i)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred, squared = False)
    if mse < lowest:
        lowest = mse
        print("random = ", i, "mse = ", mse)
        tx = dTest.drop(['transmission_label'], axis=1)
        tx = sm.add_constant(tx)
        y_pred_sf = model.predict(tx)
        submission_df = pd.DataFrame({
            'id': id,
            'class': y_pred_sf
        })
        submission_df.to_csv('submission_file.csv', index=False)

    

random =  1 mse =  68381.5428644021
random =  36 mse =  65118.66478896381
random =  530 mse =  64615.080126977315
random =  534 mse =  63673.48378195095
random =  3000 mse =  63126.48158816685
random =  3769 mse =  61938.639897929555
random =  3935 mse =  61051.34505543762
random =  12330 mse =  60013.91177110974


KeyboardInterrupt: 

In [100]:
dTest

,milage,accident,horse_power,engine_capacity,model_age,brand_label,fuel_type_label,ext_col_label,int_col_label,transmission_label
0,98000,False,240.0,2.0,9,26,2,302,10,16
1,9142,False,395.0,3.0,4,26,3,261,14,31
2,28121,False,None,3.5,2,14,2,302,57,3
3,61258,False,None,None,8,3,2,259,14,39
4,59000,False,252.0,2.0,6,3,2,127,14,38
...,...,...,...,...,...,...,...,...,...,...
125685,83315,False,362.0,3.0,10,36,2,29,14,23
125686,29336,False,None,None,5,3,2,302,14,39
125687,77634,False,333.0,3.0,12,3,2,29,14,38
125688,112000,False,333.0,3.0,12,3,2,29,14,38
